# Feature Engineering Validation

This notebook validates the output of the feature engineering process for financial data. It performs comprehensive data quality checks including:

- **Schema Validation**: Ensures all expected columns are present
- **Null Value Analysis**: Identifies missing data and completeness metrics
- **Outlier Detection**: Detects extreme values in daily returns and other features
- **Data Quality Summary**: Provides overall assessment of feature tables

## Expected Validation Results:
- ✅ No missing feature columns
- ✅ Null counts minimal (< 5% of total records)
- ✅ Outlier count reasonable (< 1% extreme outliers)

## 1. Import Required Libraries

Import PySpark SQL functions and other necessary libraries for data validation.

In [ ]:
# Databricks notebook source
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")
print(f"Validation started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Define Stock Symbols and Table Names

Set up the list of stock symbols to validate and construct corresponding table names.

In [ ]:
# Define symbols to validate
symbols = ["AAPL", "MSFT"]

# Expected feature columns
expected_columns = [
    "ticker", "date", "open", "high", "low", "close", "volume",
    "daily_return", "moving_avg_7", "moving_avg_30", "volatility_7", 
    "momentum", "feature_timestamp"
]

# Create table mapping
table_mapping = {}
for symbol in symbols:
    table_name = f"main.finance.features_{symbol.lower()}"
    table_mapping[symbol] = table_name

print("🎯 Validation Configuration:")
print(f"   Symbols to validate: {', '.join(symbols)}")
print(f"   Expected columns: {len(expected_columns)}")
print(f"   Table mapping:")
for symbol, table in table_mapping.items():
    print(f"     {symbol} → {table}")

## 3. Load and Validate Feature Tables

Load each feature table using Spark and display the schema to verify column structure.

In [ ]:
# Load and validate each feature table
dataframes = {}
validation_results = {}

print("📊 Loading Feature Tables and Validating Schemas")
print("=" * 60)

for symbol in symbols:
    table_name = table_mapping[symbol]
    print(f"\n🏷️ Validating {table_name}")
    
    try:
        # Check if table exists
        if spark.catalog.tableExists(table_name):
            # Load the table
            df = spark.table(table_name)
            dataframes[symbol] = df
            
            print(f"✅ Table exists: {table_name}")
            print(f"📊 Row count: {df.count():,}")
            print(f"📋 Column count: {len(df.columns)}")
            
            # Display schema
            print("📝 Table Schema:")
            df.printSchema()
            
            # Initialize validation results
            validation_results[symbol] = {
                'table_exists': True,
                'row_count': df.count(),
                'column_count': len(df.columns),
                'columns': df.columns
            }
            
        else:
            print(f"❌ Table does not exist: {table_name}")
            validation_results[symbol] = {
                'table_exists': False,
                'error': f"Table {table_name} not found"
            }
            
    except Exception as e:
        print(f"❌ Error accessing table {table_name}: {str(e)}")
        validation_results[symbol] = {
            'table_exists': False,
            'error': str(e)
        }

## 4. Check for Missing Columns

Validate that all expected feature columns are present in each table.

In [ ]:
# Check for missing columns in each table
print("🔍 Column Validation Analysis")
print("=" * 50)

for symbol in symbols:
    if symbol in dataframes:
        df = dataframes[symbol]
        actual_columns = set(df.columns)
        expected_columns_set = set(expected_columns)
        
        print(f"\n🏷️ {symbol} Column Analysis:")
        print(f"   Expected columns: {len(expected_columns_set)}")
        print(f"   Actual columns: {len(actual_columns)}")
        
        # Find missing columns
        missing_columns = expected_columns_set - actual_columns
        extra_columns = actual_columns - expected_columns_set
        
        if missing_columns:
            print(f"   ❌ Missing columns: {', '.join(missing_columns)}")
            validation_results[symbol]['missing_columns'] = list(missing_columns)
        else:
            print(f"   ✅ All expected columns present")
            validation_results[symbol]['missing_columns'] = []
        
        if extra_columns:
            print(f"   ⚠️ Extra columns: {', '.join(extra_columns)}")
            validation_results[symbol]['extra_columns'] = list(extra_columns)
        else:
            validation_results[symbol]['extra_columns'] = []
            
        # Overall column validation status
        column_validation_passed = len(missing_columns) == 0
        validation_results[symbol]['column_validation_passed'] = column_validation_passed
        
        status = "✅ PASSED" if column_validation_passed else "❌ FAILED"
        print(f"   📊 Column Validation: {status}")
        
    else:
        print(f"\n🏷️ {symbol}: ❌ Table not available for column validation")

## 5. Analyze Null Value Counts

Count null values in each column to ensure data completeness and identify potential data quality issues.

In [ ]:
# Analyze null values for each symbol
print("🔍 Null Value Analysis")
print("=" * 50)

for symbol in symbols:
    if symbol in dataframes:
        df = dataframes[symbol]
        table_name = table_mapping[symbol]
        
        print(f"\n🏷️ {symbol} Null Analysis ({table_name}):")
        
        # Check for nulls using the provided pattern
        null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])
        
        # Display null counts (using display for Databricks)
        print("📊 Null Counts by Column:")
        display(null_counts)
        
        # Collect null count data for analysis
        null_data = null_counts.collect()[0].asDict()
        total_rows = df.count()
        
        # Analyze null percentages
        null_analysis = {}
        high_null_columns = []
        
        print(f"\n📈 Null Percentage Analysis (Total rows: {total_rows:,}):")
        for col, null_count in null_data.items():
            null_percentage = (null_count / total_rows * 100) if total_rows > 0 else 0
            null_analysis[col] = {
                'count': null_count,
                'percentage': null_percentage
            }
            
            # Flag columns with high null percentages
            if null_percentage > 5.0:  # More than 5% nulls
                high_null_columns.append(col)
                status = "❌"
            elif null_percentage > 1.0:  # More than 1% nulls
                status = "⚠️"
            else:
                status = "✅"
            
            print(f"   {status} {col}: {null_count:,} nulls ({null_percentage:.2f}%)")
        
        # Store results
        validation_results[symbol]['null_analysis'] = null_analysis
        validation_results[symbol]['high_null_columns'] = high_null_columns
        validation_results[symbol]['null_validation_passed'] = len(high_null_columns) == 0
        
        if high_null_columns:
            print(f"   ⚠️ Columns with high null rates (>5%): {', '.join(high_null_columns)}")
        else:
            print(f"   ✅ All columns have acceptable null rates (<5%)")
            
    else:
        print(f"\n🏷️ {symbol}: ❌ Table not available for null analysis")

## 6. Detect and Count Outliers

Identify outliers in daily returns using threshold filters and count occurrences for each symbol.

In [ ]:
# Detect outliers in daily returns and other features
print("🔍 Outlier Detection Analysis")
print("=" * 50)

# Define outlier thresholds
outlier_thresholds = {
    'daily_return': {'extreme': 0.2, 'moderate': 0.1},  # ±20% and ±10%
    'volatility_7': {'extreme': 0.5, 'moderate': 0.3},   # High volatility thresholds
    'momentum': {'extreme': 2.0, 'moderate': 1.5}        # Momentum ratio thresholds
}

for symbol in symbols:
    if symbol in dataframes:
        df = dataframes[symbol]
        table_name = table_mapping[symbol]
        
        print(f"\n🏷️ {symbol} Outlier Analysis ({table_name}):")
        
        outlier_results = {}
        
        # Analyze each feature for outliers
        for feature, thresholds in outlier_thresholds.items():
            if feature in df.columns:
                extreme_threshold = thresholds['extreme']
                moderate_threshold = thresholds['moderate']
                
                # Count extreme outliers (using the original pattern)
                if feature == 'daily_return':
                    extreme_outliers = df.filter(
                        (F.col(feature) < -extreme_threshold) | 
                        (F.col(feature) > extreme_threshold)
                    )
                    moderate_outliers = df.filter(
                        (F.col(feature) < -moderate_threshold) | 
                        (F.col(feature) > moderate_threshold)
                    )
                else:
                    # For other features, only check upper bounds
                    extreme_outliers = df.filter(F.col(feature) > extreme_threshold)
                    moderate_outliers = df.filter(F.col(feature) > moderate_threshold)
                
                extreme_count = extreme_outliers.count()
                moderate_count = moderate_outliers.count()
                total_rows = df.count()
                
                extreme_percentage = (extreme_count / total_rows * 100) if total_rows > 0 else 0
                moderate_percentage = (moderate_count / total_rows * 100) if total_rows > 0 else 0
                
                print(f"   📊 {feature}:")
                print(f"      Extreme outliers (±{extreme_threshold}): {extreme_count:,} ({extreme_percentage:.2f}%)")
                print(f"      Moderate outliers (±{moderate_threshold}): {moderate_count:,} ({moderate_percentage:.2f}%)")
                
                # Store results
                outlier_results[feature] = {
                    'extreme_count': extreme_count,
                    'extreme_percentage': extreme_percentage,
                    'moderate_count': moderate_count,
                    'moderate_percentage': moderate_percentage
                }
                
                # Flag if outlier rate is too high (>1% extreme outliers)
                if extreme_percentage > 1.0:
                    print(f"      ⚠️ High extreme outlier rate: {extreme_percentage:.2f}%")
                else:
                    print(f"      ✅ Acceptable outlier rate: {extreme_percentage:.2f}%")
        
        # Original pattern for daily_return outliers
        outliers = df.filter((F.col("daily_return") < -0.2) | (F.col("daily_return") > 0.2))
        outlier_count = outliers.count()
        print(f"\n📈 Outliers in {table_name}: {outlier_count}")
        
        # Display sample outliers if any exist
        if outlier_count > 0 and outlier_count <= 20:
            print(f"📋 Sample outlier records:")
            outliers.select("date", "ticker", "daily_return", "close").orderBy(F.desc("daily_return")).show(10)
        
        # Store validation results
        validation_results[symbol]['outlier_analysis'] = outlier_results
        validation_results[symbol]['total_extreme_outliers'] = outlier_count
        validation_results[symbol]['outlier_validation_passed'] = (outlier_count / df.count() * 100) < 1.0  # <1% outliers
        
    else:
        print(f"\n🏷️ {symbol}: ❌ Table not available for outlier analysis")

## 7. Generate Validation Summary Report

Create a comprehensive summary report showing validation results including missing columns, null counts, and outlier statistics.

In [ ]:
# Generate comprehensive validation summary report
print("📋 FEATURE VALIDATION SUMMARY REPORT")
print("=" * 60)
print(f"Validation completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Symbols validated: {', '.join(symbols)}")

# Overall validation status
all_validations_passed = True
summary_stats = {
    'tables_validated': 0,
    'tables_passed': 0,
    'total_rows': 0,
    'total_columns': 0,
    'total_outliers': 0
}

print(f"\n📊 VALIDATION RESULTS BY SYMBOL:")
print("-" * 40)

for symbol in symbols:
    if symbol in validation_results and validation_results[symbol].get('table_exists', False):
        result = validation_results[symbol]
        
        # Individual validation checks
        table_exists = result.get('table_exists', False)
        column_validation = result.get('column_validation_passed', False)
        null_validation = result.get('null_validation_passed', False) 
        outlier_validation = result.get('outlier_validation_passed', False)
        
        # Overall symbol validation
        symbol_passed = all([table_exists, column_validation, null_validation, outlier_validation])
        status_emoji = "✅" if symbol_passed else "❌"
        
        print(f"\n{status_emoji} {symbol} ({table_mapping[symbol]}):")
        print(f"   📊 Rows: {result.get('row_count', 0):,}")
        print(f"   📋 Columns: {result.get('column_count', 0)}")
        print(f"   🔍 Schema: {'✅ PASS' if column_validation else '❌ FAIL'}")
        print(f"   🔢 Nulls: {'✅ PASS' if null_validation else '❌ FAIL'}")
        print(f"   📈 Outliers: {'✅ PASS' if outlier_validation else '❌ FAIL'}")
        
        # Detailed issues if any
        if result.get('missing_columns'):
            print(f"      ⚠️ Missing columns: {', '.join(result['missing_columns'])}")
        if result.get('high_null_columns'):
            print(f"      ⚠️ High null columns: {', '.join(result['high_null_columns'])}")
        if result.get('total_extreme_outliers', 0) > 0:
            outlier_pct = (result['total_extreme_outliers'] / result.get('row_count', 1)) * 100
            print(f"      ⚠️ Extreme outliers: {result['total_extreme_outliers']} ({outlier_pct:.2f}%)")
        
        # Update summary stats
        summary_stats['tables_validated'] += 1
        if symbol_passed:
            summary_stats['tables_passed'] += 1
        summary_stats['total_rows'] += result.get('row_count', 0)
        summary_stats['total_columns'] += result.get('column_count', 0)
        summary_stats['total_outliers'] += result.get('total_extreme_outliers', 0)
        
        if not symbol_passed:
            all_validations_passed = False
            
    else:
        print(f"\n❌ {symbol}: Table not found or inaccessible")
        all_validations_passed = False

# Final summary
print(f"\n🎯 OVERALL VALIDATION SUMMARY:")
print("-" * 30)
print(f"📊 Tables validated: {summary_stats['tables_validated']}")
print(f"✅ Tables passed: {summary_stats['tables_passed']}")
print(f"❌ Tables failed: {summary_stats['tables_validated'] - summary_stats['tables_passed']}")
print(f"📈 Total data rows: {summary_stats['total_rows']:,}")
print(f"📋 Average columns per table: {summary_stats['total_columns'] / max(summary_stats['tables_validated'], 1):.1f}")
print(f"⚠️ Total extreme outliers: {summary_stats['total_outliers']:,}")

# Overall status
if all_validations_passed:
    print(f"\n🎉 VALIDATION RESULT: ✅ ALL CHECKS PASSED")
    print(f"   • ✅ No missing feature columns")
    print(f"   • ✅ Null counts minimal")  
    print(f"   • ✅ Outlier count reasonable")
else:
    print(f"\n⚠️ VALIDATION RESULT: ❌ SOME CHECKS FAILED")
    print(f"   Please review the detailed results above")

print(f"\n📝 Next Steps:")
if all_validations_passed:
    print(f"   1. ✅ Feature tables are ready for ML model training")
    print(f"   2. 🚀 Proceed with model development workflows")
    print(f"   3. 📊 Set up monitoring for ongoing data quality")
else:
    print(f"   1. 🔧 Address the validation failures identified above")
    print(f"   2. 🔄 Re-run feature engineering if necessary")
    print(f"   3. 🔍 Investigate data quality issues")

print(f"\n✨ Validation completed successfully!")